# 1. SETTINGS

In [8]:
# libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.metrics import roc_auc_score
import scipy.stats

In [9]:
# pandas options
pd.set_option("display.max_columns", None)

# 2. DATA PARTITIONING

In [45]:
# import data
data = pd.read_csv("../../data/prepared/train_no_holding.csv")

In [11]:
data.CustomerInterest.mean()

0.015928279254888048

In [12]:
# check all datasets
print("Test data:", test.shape)
display(test.head(3))
print("------------------------------")
print("Train data:", data.shape)
display(data.head(3))

NameError: name 'test' is not defined

In [46]:
# data partitioning
stats = data[(data["Week"] >= 1)   & (data["Week"] <= 50)]
train = data[(data["Week"] >= 51)  & (data["Week"] <= 110)]
valid = data[(data["Week"] >= 111) & (data["Week"] <= 121)]
del data

# 3. COMPUTING NAIVE RATIOS (TRAIN)

In [47]:
# compute target ratio (last 64 weeks)
cust_int0 = train[train["Week"] >= (110-64)]
cust_int0 = cust_int0[["CustomerIdx", "CustomerInterest", "IsinIdx"]]
cust_int0 = cust_int0.groupby(["CustomerIdx", "IsinIdx"], as_index = False).mean()
cust_int0.columns = ["CustomerIdx", "IsinIdx", "ratio0"]

In [48]:
# compute target ratio (last 32 weeks)
cust_int1 = train[train["Week"] >= (110-32)]
cust_int1 = cust_int1[["CustomerIdx", "CustomerInterest", "IsinIdx"]]
cust_int1 = cust_int1.groupby(["CustomerIdx", "IsinIdx"], as_index = False).mean()
cust_int1.columns = ["CustomerIdx", "IsinIdx", "ratio1"]

In [49]:
# compute target ratio (last 16 weeks)
cust_int2 = train[train["Week"] >= (110-16)]
cust_int2 = cust_int2[["CustomerIdx", "CustomerInterest", "IsinIdx"]]
cust_int2 = cust_int2.groupby(["CustomerIdx", "IsinIdx"], as_index = False).mean()
cust_int2.columns = ["CustomerIdx", "IsinIdx", "ratio2"]

In [50]:
# compute target ratio (last 8 weeks)
cust_int3 = train[train["Week"] >= (110-8)]
cust_int3 = cust_int3[["CustomerIdx", "CustomerInterest", "IsinIdx"]]
cust_int3 = cust_int3.groupby(["CustomerIdx", "IsinIdx"], as_index = False).mean()
cust_int3.columns = ["CustomerIdx", "IsinIdx", "ratio3"]

In [51]:
# compute target ratio (last 4 weeks)
cust_int4 = train[train["Week"] >= (110-4)]
cust_int4 = cust_int4[["CustomerIdx", "CustomerInterest", "IsinIdx"]]
cust_int4 = cust_int4.groupby(["CustomerIdx", "IsinIdx"], as_index = False).mean()
cust_int4.columns = ["CustomerIdx", "IsinIdx", "ratio4"]

In [52]:
# compute customer target ratio (last 32 weeks)
cust_int5 = train[train["Week"] >= (110-32)]
cust_int5 = cust_int5[["CustomerIdx", "CustomerInterest"]]
cust_int5 = cust_int5.groupby(["CustomerIdx"], as_index = False).mean()
cust_int5.columns = ["CustomerIdx", "ratio5"]

In [53]:
# compute bond target ratio (last 32 weeks)
cust_int6 = train[train["Week"] >= (110-32)]
cust_int6 = cust_int6[["IsinIdx", "CustomerInterest"]]
cust_int6 = cust_int6.groupby(["IsinIdx"], as_index = False).mean()
cust_int6.columns = ["IsinIdx", "ratio6"]

In [54]:
# merge and average all ratios
cust_int = cust_int0.merge(cust_int1, how = "left", on = ["CustomerIdx", "IsinIdx"])
cust_int = cust_int.merge(cust_int2,  how = "left", on = ["CustomerIdx", "IsinIdx"])
cust_int = cust_int.merge(cust_int3,  how = "left", on = ["CustomerIdx", "IsinIdx"])
cust_int = cust_int.merge(cust_int4,  how = "left", on = ["CustomerIdx", "IsinIdx"])
cust_int = cust_int.merge(cust_int5,  how = "left", on = ["CustomerIdx"])
cust_int = cust_int.merge(cust_int6,  how = "left", on = ["IsinIdx"])
cust_int["RatioMean"] = (cust_int["ratio0"] + cust_int["ratio1"] + cust_int["ratio2"] + 
                         cust_int["ratio3"] + cust_int["ratio4"] + cust_int["ratio5"] +
                         cust_int["ratio6"]) / 7
cust_int.head()

,CustomerIdx,IsinIdx,ratio0,ratio1,ratio2,ratio3,ratio4,ratio5,ratio6,RatioMean
0,0,16471,0.016667,0.000000,0.0,0.0,0.0,0.007346,0.000000,0.003430
1,0,16499,0.016667,0.000000,0.0,0.0,0.0,0.007346,0.000000,0.003430
2,0,20601,0.016667,0.000000,0.0,0.0,0.0,0.007346,0.002886,0.003843
3,0,20737,0.016667,0.030303,0.0,0.0,0.0,0.007346,0.016414,0.010104
4,0,20909,0.000000,0.000000,0.0,0.0,0.0,0.007346,0.000000,0.001049


In [55]:
# merge to valid
valid = valid.merge(cust_int, how = "left", on = ["CustomerIdx", "IsinIdx"])
valid.head()

,CustomerIdx,IsinIdx,BuySell,Week,CustomerInterest,ratio0,ratio1,ratio2,ratio3,ratio4,ratio5,ratio6,RatioMean
0,0,16471,Buy,111.0,0,0.016667,0.0,0.0,0.0,0.0,0.007346,0.0,0.00343
1,0,16471,Buy,112.0,0,0.016667,0.0,0.0,0.0,0.0,0.007346,0.0,0.00343
2,0,16471,Buy,113.0,0,0.016667,0.0,0.0,0.0,0.0,0.007346,0.0,0.00343
3,0,16471,Buy,114.0,0,0.016667,0.0,0.0,0.0,0.0,0.007346,0.0,0.00343
4,0,16471,Buy,115.0,0,0.016667,0.0,0.0,0.0,0.0,0.007346,0.0,0.00343


In [56]:
# compute AUC ROC
features = ["ratio1", "ratio2", "ratio3", "ratio4", "ratio5", "ratio6", "RatioMean"]
for r in features:
    print(roc_auc_score(valid.CustomerInterest, valid[r]))

0.5412587412855547
0.5633154307271843
0.5659050526501382
0.560958338546176
0.603797771730836
0.5802206157460548
0.5556984885908877


In [57]:
y = train.CustomerInterest

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [59]:
### PARAMETERS

import lightgbm as lgb

# learner settings
metric   = "auc"
verbose  = 10
stopping = 50
seed = 42

# lgb settings
gbm = lgb.LGBMClassifier(n_estimators     = 1000,
                         learning_rate    = 0.005,
                         num_leaves       = 70,
                         colsample_bytree = 0.8,
                         subsample        = 0.9,
                         max_depth        = 7,
                         reg_alpha        = 0.1,
                         reg_lambda       = 0.1,
                         min_split_gain   = 0.01,
                         min_child_weight = 2,
                         random_state     = seed)

In [29]:
# train lightGBM
gbm = gbm.fit(train[features], y_train, 
              eval_set = [(train[features], y_train), (valid[features], y_valid)], 
              eval_metric = metric, 
              verbose = verbose, 
              early_stopping_rounds = stopping)
    
# save number of iterations
num_iters = gbm.best_iteration_

# predict valid and test
valid_pred_CustomerInterest = gbm.predict_proba(valid[features], num_iteration = num_iters)[:, 1]

KeyError: "['ratio1' 'ratio2' 'ratio3' 'ratio4' 'ratio5' 'ratio6' 'RatioMean'] not in index"

In [ ]:
ax = lgb.plot_importance(gbm)
plt.show()

# 4. COMPUTING NAIVE RATIOS (FULL)

In [60]:
# import data
#train = pd.read_csv("../../data/prepared/train_no_holding.csv")
test  = pd.read_csv("../../data/prepared/test_no_holding.csv")

In [36]:
# compute target ratio (last 64 weeks)
cust_int0 = train[train["Week"] >= (121-64)]
cust_int0 = cust_int0[["CustomerIdx", "CustomerInterest", "IsinIdx"]]
cust_int0 = cust_int0.groupby(["CustomerIdx", "IsinIdx"], as_index = False).mean()
cust_int0.columns = ["CustomerIdx", "IsinIdx", "ratio0"]

In [37]:
# compute target ratio (last 32 weeks)
cust_int1 = train[train["Week"] >= (121-32)]
cust_int1 = cust_int1[["CustomerIdx", "CustomerInterest", "IsinIdx"]]
cust_int1 = cust_int1.groupby(["CustomerIdx", "IsinIdx"], as_index = False).mean()
cust_int1.columns = ["CustomerIdx", "IsinIdx", "ratio1"]

In [38]:
# compute target ratio (last 16 weeks)
cust_int2 = train[train["Week"] >= (121-16)]
cust_int2 = cust_int2[["CustomerIdx", "CustomerInterest", "IsinIdx"]]
cust_int2 = cust_int2.groupby(["CustomerIdx", "IsinIdx"], as_index = False).mean()
cust_int2.columns = ["CustomerIdx", "IsinIdx", "ratio2"]

In [39]:
# compute target ratio (last 8 weeks)
cust_int3 = train[train["Week"] >= (121-8)]
cust_int3 = cust_int3[["CustomerIdx", "CustomerInterest", "IsinIdx"]]
cust_int3 = cust_int3.groupby(["CustomerIdx", "IsinIdx"], as_index = False).mean()
cust_int3.columns = ["CustomerIdx", "IsinIdx", "ratio3"]

In [40]:
# compute target ratio (last 4 weeks)
cust_int4 = train[train["Week"] >= (121-4)]
cust_int4 = cust_int4[["CustomerIdx", "CustomerInterest", "IsinIdx"]]
cust_int4 = cust_int4.groupby(["CustomerIdx", "IsinIdx"], as_index = False).mean()
cust_int4.columns = ["CustomerIdx", "IsinIdx", "ratio4"]

In [41]:
# compute customer target ratio (last 32 weeks)
cust_int5 = train[train["Week"] >= (121-32)]
cust_int5 = cust_int5[["CustomerIdx", "CustomerInterest"]]
cust_int5 = cust_int5.groupby(["CustomerIdx"], as_index = False).mean()
cust_int5.columns = ["CustomerIdx", "ratio5"]

In [42]:
# compute bond target ratio (last 32 weeks)
cust_int6 = train[train["Week"] >= (121-32)]
cust_int6 = cust_int6[["IsinIdx", "CustomerInterest"]]
cust_int6 = cust_int6.groupby(["IsinIdx"], as_index = False).mean()
cust_int6.columns = ["IsinIdx", "ratio6"]

In [43]:
# merge and average all ratios
cust_int = cust_int0.merge(cust_int1, how = "left", on = ["CustomerIdx", "IsinIdx"])
cust_int = cust_int.merge(cust_int2,  how = "left", on = ["CustomerIdx", "IsinIdx"])
cust_int = cust_int.merge(cust_int3,  how = "left", on = ["CustomerIdx", "IsinIdx"])
cust_int = cust_int.merge(cust_int4,  how = "left", on = ["CustomerIdx", "IsinIdx"])
cust_int = cust_int.merge(cust_int5,  how = "left", on = ["CustomerIdx"])
cust_int = cust_int.merge(cust_int6,  how = "left", on = ["IsinIdx"])
cust_int["RatioMean"] = (cust_int["ratio0"] + cust_int["ratio1"] + cust_int["ratio2"] + 
                         cust_int["ratio3"] + cust_int["ratio4"] + cust_int["ratio5"] +
                         cust_int["ratio6"]) / 7
cust_int.head()

,CustomerIdx,IsinIdx,ratio0,ratio1,ratio2,ratio3,ratio4,ratio5,ratio6,RatioMean
0,0,16471,0.015625,0.0,0.0,0.0,0.0,0.013258,0.000000,0.004126
1,0,16499,0.015625,0.0,0.0,0.0,0.0,0.013258,0.000000,0.004126
2,0,20601,0.015625,0.0,0.0,0.0,0.0,0.013258,0.000000,0.004126
3,0,20737,0.015625,0.0,0.0,0.0,0.0,0.013258,0.011719,0.005800
4,0,20909,0.000000,0.0,0.0,0.0,0.0,0.013258,0.000000,0.001894


In [61]:
# merge to test
del test["CustomerInterest"]
test = test.merge(cust_int, how = "left", on = ["CustomerIdx", "IsinIdx"])
test.head()

,PredictionIdx,DateKey,CustomerIdx,IsinIdx,BuySell,Week,ratio0,ratio1,ratio2,ratio3,ratio4,ratio5,ratio6,RatioMean
0,a1e0d80784,23042018,1856,13323,Buy,121.0,0.016667,0.030303,0.058824,0.000000,0.0,0.026928,0.011655,0.020625
1,c2cc6cc2a8,23042018,1856,9230,Buy,121.0,0.025000,0.030303,0.058824,0.055556,0.1,0.026928,0.012987,0.044228
2,a8e94f6344,23042018,1780,9157,Buy,121.0,0.033333,0.060606,0.117647,0.111111,0.0,0.017849,0.010101,0.050093
3,758bae1e35,23042018,2129,9131,Buy,121.0,0.016667,0.030303,0.058824,0.111111,0.0,0.010226,0.022282,0.035630
4,02ab378ee8,23042018,1758,7151,Buy,121.0,0.016667,0.030303,0.058824,0.000000,0.0,0.016766,0.006494,0.018436


In [62]:
# compute prediction
test["CustomerInterest"] = test["ratio6"]
test.CustomerInterest.mean()

0.02018854949597839

In [31]:
classifier = LogisticRegression(C=3)
classifier.fit(valid[features], valid.CustomerInterest)
probs = classifier.predict_proba(test[features])[:,1]

In [63]:
import lightgbm as lgb
gbm = gbm.fit(valid[features], valid.CustomerInterest, 
              eval_set = [(valid[features], valid.CustomerInterest)], 
              eval_metric = 'auc', 
              verbose = verbose,
                 early_stopping_rounds = 50)
    
# save number of iterations
num_iters = gbm.best_iteration_

# predict valid and test
valid_pred_CustomerInterest = gbm.predict_proba(test[features], num_iteration = num_iters)[:, 1]

Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.704178
[20]	valid_0's auc: 0.703033
[30]	valid_0's auc: 0.701813
[40]	valid_0's auc: 0.704412
[50]	valid_0's auc: 0.706137
[60]	valid_0's auc: 0.707312
[70]	valid_0's auc: 0.707135
[80]	valid_0's auc: 0.70671
[90]	valid_0's auc: 0.707381
[100]	valid_0's auc: 0.707909
[110]	valid_0's auc: 0.707897
[120]	valid_0's auc: 0.707768
[130]	valid_0's auc: 0.707883
[140]	valid_0's auc: 0.707653
[150]	valid_0's auc: 0.707898
Early stopping, best iteration is:
[104]	valid_0's auc: 0.708162


In [64]:
# export CSV
subm = test[["PredictionIdx", "CustomerInterest"]]
subm.CustomerInterest = valid_pred_CustomerInterest 
subm.to_csv("../../submissions/lgb_novalid.csv", index = False, float_format = "%.8f")

C:\Users\zinovyee.hub\AppData\Local\conda\conda\envs\thesis\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
